# Task 1 Simple Text-based Similarity and Retrieval

### Team E
Version 1.0.1
Date: 14/11/2023

## Import Libraries  

In [1]:
import pandas as pd
import numpy as np
import statistics as st

In [17]:
from ret import read_data # utility func to load data
from ret import get_id_from_info # utility func to return id by entering song's info
from ret import display_res # utility func to display results 
from ret import audio_based # modularized text based retrieval system
from ret import random_baseline # base line retrieval system that returns random results 
from ret import cos_sim # wrapper function to calculate cosine similarity
from ret import euc_sim # wrapper function to calculate euclidean similarity 
from ret import get_info_from_ids #utility func to return infro from id
from ret import get_genre_from_ids 
from ret import get_genre_from_query
from ret import calculate_precision 
from ret import calculate_recall
from ret import count_relevant_songs_in_dataset
from ret import count_relevant_songs_in_result
from ret import average_precision
from ret import average_recall

## Load Data

In [18]:
"""
Please put data files in ""./data/" before use 
"""

info = read_data("information")
mfcc_stats = read_data("mfcc_stats")
mfcc_bow = read_data("mfcc_bow")
genres = read_data("genres")

genres

,id,genre
0,01Yfj2T3YTwJ1Yfy,"['rock', 'christian rock']"
1,01gyRHLquwXDlhkO,"['hip hop', 'rap', 'grindcore', 'death metal']"
2,01rMxQv6vhyE1oQX,"['rock', 'pop punk']"
3,02RGE9FNH65RtMS7,"['trance', 'techno', 'progressive trance']"
4,02ZnlCGZEbkfCDxo,"['pop', 'italian pop', 'latin', 'europop', 'am..."
...,...,...
10089,zyzILCQvVeUFIINi,"['rock', 'pop', 'indie rock', 'folk rock', 'ne..."
10090,zzgS4ZqyswamEWNj,"['pop', 'rock', 'teen pop', 'soundtrack', 'sin..."
10091,zzoFYDMlqU1X2zz1,"['jazz', 'jazz fusion']"
10092,zzpkRCGA5ud8q4mv,"['soul', 'blues', 'r b', 'blues rock', 'southe..."


In [19]:
dataset_genres = genres.values.tolist()


In [20]:
def average_recall(r1, r2, r3):
    recalls = [r1,r2,r3]
    average_recall = st.mean(recalls)
    return average_recall

## Dictionary to Store Results 

In [21]:
"""
Dict to store results:
query format: res["trackTRACKNUMBER"]["REPRESENTATION"]["SIMILARITYFUNCTION"]
sample query: res["track1"]["tfidf"]["cos_sim"]

returns - list[str] 
list containing ids of a retrieved tracks 

After running, use display_res(res["trackTRACKNUMBER"]["REPRESENTATION"]["SIMILARITYFUNCTION"],info) to display results
e.g. display_res(res["track1"]["tfidf"]["cos_sim"],info)

"""
res={
    "track1": 
    {
    "base_line": None,
    "mfcc_bow":
    {
        "cos_sim": None,
        "euc_sim": None
    },
    "mfcc_stats":
    {
        "cos_sim": None,
        "euc_sim": None
    }
    },
    "track2": 
    {
    "base_line": None,
    "mfcc_bow":
    {
        "cos_sim": None,
        "euc_sim": None
    },
    "mfcc_stats":
    {
        "cos_sim": None,
        "euc_sim": None
    }
    },
    "track3": 
    {
    "base_line": None,
    "mfcc_bow":
    {
        "cos_sim": None,
        "euc_sim": None
    },
    "mfcc_stats":
    {
        "cos_sim": None,
        "euc_sim": None
    }
    
}
}

## Track 1 
"Love Me" "The 1975"

In [22]:
name = "Love Me"
artist = "The 1975"
id_track1 = get_id_from_info(name, artist, info)
id_track1

'rTXO5VOLQGcypcmB'

In [23]:
res["track1"]["mfcc_bow"]["cos_sim"] = audio_based(id=id_track1, repr=mfcc_bow, N=10, sim_func=cos_sim)

In [24]:
res["track1"]["mfcc_stats"]["cos_sim"] = audio_based(id=id_track1, repr=mfcc_stats, N=10, sim_func=cos_sim)

In [25]:
query_genre = get_genre_from_query(id_track1, genres)
retrieved_genres = get_genre_from_ids(audio_based(id=id_track1, repr=mfcc_stats, N=10, sim_func=cos_sim), genres)
query_genre

[('rTXO5VOLQGcypcmB',
  "['pop', 'rock', 'indie pop', 'electropop', 'indie rock', 'funk', 'funk rock']")]

In [26]:
p1 = calculate_precision(query_genre, retrieved_genres)
p1

0.9

In [27]:
retrieved_genres

[('XXShp3IW8QBbI6AU', "['soul', 'dance pop']"),
 ('pNUcTH7vMfVZ7yYs', "['pop', 'rain', 'europop']"),
 ('51XYIfDaAUgQXb5u',
  "['pop', 'disco', 'europop', 'rock', 'pop rock', 'classic rock', 'dance pop', 'soft rock', 'swedish pop']"),
 ('CZuApL1gKvIynMwO',
  "['pop', 'dance pop', 'electropop', 'hip hop', 'anthem']"),
 ('CMoYcyU4kMOa3sVY', "['ukulele', 'rock', 'alternative rock', 'pop rock']"),
 ('DCuWGUQBdWiIXEkR', "['pop', 'latin', 'latin pop', 'teen pop']"),
 ('2K0JDCBAlEvRXqrl',
  "['rock', 'alternative rock', 'indie rock', 'experimental', 'dub', 'indietronica', 'alternative dance', 'art pop']"),
 ('zLIZEr0zFataPTEI', "['pop', 'soul', 'singer songwriter', 'r b']"),
 ('4RsPg7cccVfV7BWQ', "['rock', 'jazz pop']"),
 ('mxKahNL0muWgijyX',
  "['pop', 'rock', 'pop rock', 'punk', 'alternative rock', 'anarcho punk', 'dance pop', 'alternative dance', 'dance punk', 'europop', 'acid house']")]

In [28]:
count_relevant_songs_in_dataset(query_genre,dataset_genres)

7187

In [29]:
count_relevant_songs_in_result(query_genre,retrieved_genres)

9

In [30]:
r1 = calculate_recall(query_genre, retrieved_genres, dataset_genres)
r1

0.001252261026854042

In [31]:
display_res(res["track1"]["mfcc_bow"]["cos_sim"], info)

Name: High Horse                               Singer: Kacey Musgraves
Name: SPRORGNSM                                Singer: Superorganism
Name: Goin' Crazy                              Singer: Ashley Tisdale
Name: Do Somethin'                             Singer: Britney Spears
Name: Ni**as In Paris                          Singer: Jay-Z & Kanye West
Name: Good... Good                             Singer: Belinda
Name: Honey, Don't Kill My Vibe                Singer: YESEO
Name: Girlfriend                               Singer: Bea Miller
Name: Jealousy                                 Singer: Paris Hilton
Name: No Time For Love                         Singer: Bryan Adams


In [32]:
display_res(res["track1"]["mfcc_stats"]["cos_sim"], info)

Name: Falling for You                          Singer: Tamia
Name: Sunshine on a Rainy Day                  Singer: Emma Bunton
Name: Honey, Honey                             Singer: ABBA
Name: Do You Wanna Come Over?                  Singer: Britney Spears
Name: The Thoughts That Give Me the Creeps     Singer: Hellogoodbye
Name: Good... Good                             Singer: Belinda
Name: Everybody's Weird                        Singer: dEUS
Name: Hesitation                               Singer: Stacie Orrico
Name: Fm                                       Singer: Nathan Haines
Name: Amnesia                                  Singer: Chumbawamba


In [33]:
display_res(res["track1"]["base_line"], info)

TypeError: 'NoneType' object is not iterable

## Track 2 

"One" "U2"

In [ ]:
name = 'One'
artist = 'U2'
id_track2 = get_id_from_info(name, artist, info)

In [ ]:
res["track2"]["mfcc_bow"]["cos_sim"] = audio_based(id=id_track2, repr=mfcc_bow, N=10, sim_func=cos_sim)

In [ ]:
res["track2"]["mfcc_stats"]["cos_sim"] = audio_based(id=id_track2, repr=mfcc_stats, N=10, sim_func=cos_sim)

In [ ]:
query_genre2 = get_genre_from_query(id_track2, genres)
retrieved_genres2 = get_genre_from_ids(audio_based(id=id_track2, repr=mfcc_stats, N=10, sim_func=cos_sim), genres)
query_genre2

In [ ]:
retrieved_genres2

In [ ]:
p2 = calculate_precision(query_genre2, retrieved_genres2)
p2

In [ ]:
r2 = calculate_recall(query_genre2, retrieved_genres2, dataset_genres)

In [ ]:
def compute_genre_distribution(query_genre, retrieved_result):
    # Get unique genres in the dataset
    all_genres = set()

    # Update genre distribution based on the query genre
    query_genres = set(eval(query_genre[0][1]))
    for genre in query_genres:
        all_genres.add(genre)

    # Initialize genre distribution vector with zeros
    genre_distribution = {genre: 0.0 for genre in all_genres}

    # Update genre distribution based on the query genre
    for genre in query_genres:
        genre_distribution[genre] += 1.0

    # Update genre distribution based on the retrieved tracks
    for _, retrieved_genres_str in retrieved_result:
        retrieved_genres = set(eval(retrieved_genres_str))
        for genre in retrieved_genres:
            if genre in all_genres:
                genre_distribution[genre] += 1.0 / len(retrieved_result)

    # Convert genre distribution to a list of tuples
    genre_distribution_list = [(genre, value) for genre, value in genre_distribution.items()]

    return genre_distribution_list

In [ ]:
compute_genre_distribution(query_genre2,retrieved_genres2)

In [ ]:
display_res(res["track2"]["mfcc_bow"]["cos_sim"], info)

In [ ]:
display_res(res["track2"]["mfcc_stats"]["cos_sim"], info)

In [ ]:
display_res(res["track2"]["base_line"], info)

## Track 3

"Every Christmas" "Kelly Clarkson"

In [ ]:
name = 'Every Christmas'
artist = 'Kelly Clarkson'
id_track3 = get_id_from_info(name, artist, info)

In [ ]:
res["track3"]["mfcc_bow"]["cos_sim"] = audio_based(id=id_track3, repr=mfcc_bow, N=10, sim_func=cos_sim)

In [ ]:
res["track3"]["mfcc_stats"]["cos_sim"] = audio_based(id=id_track3, repr=mfcc_stats, N=10, sim_func=cos_sim)

In [ ]:
res["track3"]["base_line"] = random_baseline(id=id_track3, info=info, N=10)

In [ ]:
query_genre3 = get_genre_from_query(id_track3, genres)
query_genre3

In [ ]:
retrieved_genres3 = get_genre_from_ids(audio_based(id=id_track3, repr=mfcc_stats, N=10, sim_func=cos_sim), genres)
retrieved_genres3

In [ ]:
p3 = calculate_precision(query_genre3, retrieved_genres3)
p3

In [ ]:
r3 = calculate_recall(query_genre3, retrieved_genres3, dataset_genres)
r3

In [ ]:
display_res(res["track3"]["mfcc_bow"]["cos_sim"], info)

In [ ]:
display_res(res["track3"]["mfcc_stats"]["cos_sim"], info)

In [ ]:
display_res(res["track3"]["base_line"], info)

# Accuracy

In [ ]:
average_precision(p1, p2, p3)

In [ ]:
average_recall(r1, r2, r3)